In [32]:
from keras.models import load_model
import numpy as np
from keras.layers import concatenate, Input
import tensorflow as tf
from keras.models import Model
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [2]:
# Load the saved models
alexnet = load_model('../models/alexnet_0_ultrasound')
fcn = load_model('../models/fcn_0_ultrasound')
lenet=load_model('/notebooks/ensemble_nns_major/models/lenet5_0_ultrasound')

In [3]:
image_shape=(227,227)
train_dir="/notebooks/ultrasound/train"
validation_dir=train_dir
test_dir="/notebooks/ultrasound/val"
batch_size=64
test_train_split=0.3

test_data = image_dataset_from_directory(test_dir,
      color_mode="grayscale",image_size=image_shape, batch_size=batch_size)


Found 900 files belonging to 2 classes.


In [23]:
fcn_predict=fcn.predict(test_data,verbose=0).reshape((900,))
alexnet_predict=alexnet.predict(test_data,verbose=0).reshape((900,))
lenet_predict=lenet.predict(test_data,verbose=0).reshape((900,))

In [5]:
op=np.array([])
for i,j in test_data:
    op=np.append(op,j.numpy())

In [6]:
op.shape

(900,)

In [7]:
fcn_predict.shape

(900,)

In [16]:
results = fcn.evaluate(test_data)
print("test loss, test acc:", results)

15/15 [==============================] - 0s 23ms/step - loss: 0.5021 - binary_accuracy: 0.7400
test loss, test acc: [0.502132773399353, 0.7400000095367432]


In [24]:
ensemble_predictions = np.round((fcn_predict + alexnet_predict+lenet_predict) / 3)

# Evaluate accuracy of ensemble predictions
y_test = op# load your test labels here
# ensemble_accuracy = model1.evaluate(X_test, y_test, verbose=0)[1]
# print("Ensemble accuracy: {:.2f}%".format(ensemble_accuracy * 100))

In [18]:
def calculate_metrics(y_true, y_pred):
    """
    Calculate accuracy, sensitivity, and specificity.
    """
    # Define TP, FP, TN, and FN
    TP = np.sum((y_true == 1) & (y_pred == 1))
    FP = np.sum((y_true == 0) & (y_pred == 1))
    TN = np.sum((y_true == 0) & (y_pred == 0))
    FN = np.sum((y_true == 1) & (y_pred == 0))
    
    # Calculate metrics
    accuracy = (TP + TN) / (TP + FP + TN + FN)
    sensitivity = TP / (TP + FN)
    specificity = TN / (TN + FP)
    
    return accuracy, sensitivity, specificity

In [25]:
accuracy, sensitivity, specificity = calculate_metrics(y_test, alexnet_predict)

print("Accuracy:", accuracy)
print("Sensitivity:", sensitivity)
print("Specificity:", specificity)

Accuracy: 0.48
Sensitivity: 0.48
Specificity: 0.48


In [26]:
accuracy, sensitivity, specificity = calculate_metrics(y_test, lenet_predict)

print("Accuracy:", accuracy)
print("Sensitivity:", sensitivity)
print("Specificity:", specificity)

Accuracy: 0.2727272727272727
Sensitivity: 1.0
Specificity: 0.0


In [27]:
accuracy, sensitivity, specificity = calculate_metrics(y_test, fcn_predict)

print("Accuracy:", accuracy)
print("Sensitivity:", sensitivity)
print("Specificity:", specificity)

Accuracy: nan
Sensitivity: nan
Specificity: nan


/tmp/ipykernel_32/2553318004.py:12: RuntimeWarning: invalid value encountered in long_scalars
  accuracy = (TP + TN) / (TP + FP + TN + FN)
/tmp/ipykernel_32/2553318004.py:13: RuntimeWarning: invalid value encountered in long_scalars
  sensitivity = TP / (TP + FN)
/tmp/ipykernel_32/2553318004.py:14: RuntimeWarning: invalid value encountered in long_scalars
  specificity = TN / (TN + FP)


In [28]:
accuracy, sensitivity, specificity = calculate_metrics(y_test, ensemble_predictions)

print("Accuracy:", accuracy)
print("Sensitivity:", sensitivity)
print("Specificity:", specificity)

Accuracy: 0.5033333333333333
Sensitivity: 0.345
Specificity: 0.63
